## **Instalação de Pacotes**

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=c34036c8ad618820b8ba213e052468e4a5d8af031bb9edc0ca9bcb7a31f82df7
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


## **Importação dos Pacotes**

In [56]:
import string
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max, when, isnan
from pyspark.sql.functions import upper, col
from pyspark.sql import functions as sf
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

## **Inicia a sessão do Spark**

In [4]:
spark = SparkSession.builder.appName("AtividadeFinal").master('local[*]').getOrCreate()

## **Carrega os arquivos CSVs e adicona o encode para permitir acentuação**

In [7]:
idh = "/content/sample_data/idh_2010_tratado.csv"
enem = "/content/sample_data/enem_2022_tratado.csv"

In [8]:
df_i = spark.read.options(delimiter=";", header=True).option("encoding", "ISO-8859-1").csv(idh)
df_e = spark.read.options(delimiter=";", header=True).option("encoding", "ISO-8859-1").csv(enem)

## **Tratamento dos Dataframes**

In [17]:
df_i.show()

+---------------+---+
|   NM_MUNICIPIO|IDH|
+---------------+---+
|ABADIA DE GOIÁS|708|
|      ABADIÂNIA|689|
|     ABAETETUBA|628|
|        ABAIARA|628|
|          ABARÉ|575|
|         ABATIÁ|687|
|         ABAÍRA|603|
|  ABDON BATISTA|694|
|ABEL FIGUEIREDO|622|
|   ABELARDO LUZ|696|
|   ABREU E LIMA|679|
|    ABREULÂNDIA|665|
|      ACAJUTIBA|582|
|        ACARAPE|606|
|         ACARAÚ|601|
|          ACARI|679|
|          ACARÁ|506|
|          ACAUÃ|528|
|         ACEGUÁ|687|
|       ACOPIARA|595|
+---------------+---+
only showing top 20 rows



In [18]:
### Exclusão de campos que não serão usados
df_i = df_i.drop(*('NM_FEDERACAO','IDHM'))

In [19]:
df_i.show()

+---------------+---+
|   NM_MUNICIPIO|IDH|
+---------------+---+
|ABADIA DE GOIÁS|708|
|      ABADIÂNIA|689|
|     ABAETETUBA|628|
|        ABAIARA|628|
|          ABARÉ|575|
|         ABATIÁ|687|
|         ABAÍRA|603|
|  ABDON BATISTA|694|
|ABEL FIGUEIREDO|622|
|   ABELARDO LUZ|696|
|   ABREU E LIMA|679|
|    ABREULÂNDIA|665|
|      ACAJUTIBA|582|
|        ACARAPE|606|
|         ACARAÚ|601|
|          ACARI|679|
|          ACARÁ|506|
|          ACAUÃ|528|
|         ACEGUÁ|687|
|       ACOPIARA|595|
+---------------+---+
only showing top 20 rows



In [20]:
### Contagem de registros do Dataframe IDH antes de excluir os municípios duplicados
df_i.count()

5329

In [21]:
### Exclusão dos registros de municípios duplicados
df_i = df_i.dropDuplicates(['NM_MUNICIPIO'])

In [22]:
### Nova contagem dos registros de IDH
df_i.count()

5329

In [23]:
### Dataframe com dados do Enem
df_e.show()

+------------+-------+-----------+---------+------------+------------------+--------------------+-----------+----------+---------------+
|NU_INSCRICAO|TP_SEXO|TP_COR_RACA|TP_ESCOLA|IN_TREINEIRO|CO_MUNICIPIO_PROVA|  NO_MUNICIPIO_PROVA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|
+------------+-------+-----------+---------+------------+------------------+--------------------+-----------+----------+---------------+
|    2,10E+11|      M|          2|        1|           0|           5300108|            Brasília|         DF|      NULL|           NULL|
|    2,10E+11|      M|          1|        1|           0|           5300108|            Brasília|         DF|      NULL|           NULL|
|    2,10E+11|      F|          2|        1|           0|           2925758|Presidente Tancre...|         BA|       565|            760|
|    2,10E+11|      M|          3|        1|           0|           3201308|           Cariacica|         ES|       416|            320|
|    2,10E+11|      M|          3|       

In [30]:
from pyspark.sql.functions import upper, col
from pyspark.sql import functions as sf

In [31]:
### Colocar emletras maiúsculas o campo 'Municípios' do Dataframe Enem
df_e = df_e.withColumn("NO_MUNICIPIO", upper(df_e.NO_MUNICIPIO_PROVA))

### Atriubuir um identificador único para cada registro do Dataframe Enem
df_e = df_e.select("*").withColumn("id", sf.monotonically_increasing_id())

### Exclusão dos campos que não serão usados
df_e = df_e.drop(*('NU_INSCRICAO', 'CO_MUNICIPIO_PROVA', 'NO_MUNICIPIO_PROVA', 'IN_TREINEIRO'))

In [34]:
df_e.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+
|      M|          2|        1|         DF|      NULL|           NULL|            BRASÍLIA|  0|
|      M|          1|        1|         DF|      NULL|           NULL|            BRASÍLIA|  1|
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2|
|      M|          3|        1|         ES|       416|            320|           CARIACICA|  3|
|      M|          3|        1|         PA|      NULL|           NULL|         SALINÓPOLIS|  4|
|      F|          3|        1|         RJ|      NULL|           NULL|      RIO DE JANEIRO|  5|
|      M|          2|        1|         PE|      NULL|           NULL|              RECIFE|  6|
|      F|          2|        1|         

In [35]:
### Join dos Datafrmaes IDH e Enem pelo Município
df = df_e.join(df_i, df_e.NO_MUNICIPIO == df_i.NM_MUNICIPIO, "left" )
df = df.drop('NM_MUNICIPIO')

In [36]:
df.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+----+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id| IDH|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+----+
|      M|          2|        1|         DF|      NULL|           NULL|            BRASÍLIA|  0| 824|
|      M|          1|        1|         DF|      NULL|           NULL|            BRASÍLIA|  1| 824|
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2| 559|
|      M|          3|        1|         ES|       416|            320|           CARIACICA|  3| 718|
|      M|          3|        1|         PA|      NULL|           NULL|         SALINÓPOLIS|  4| 647|
|      F|          3|        1|         RJ|      NULL|           NULL|      RIO DE JANEIRO|  5| 799|
|      M|          2|        1|         PE|      NULL|           NULL|              RECIFE|

In [37]:
### Contagem de registros do Dataframe Enem antes da exlusão dos registros sem nota de Matemática ou Redação
df.count()

1048575

In [38]:
### Exclusão dos registros nulos de notas de Matemática e Redação
df = df.na.drop(how="any")

In [39]:
df.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id|IDH|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2|559|
|      M|          3|        1|         ES|       416|            320|           CARIACICA|  3|718|
|      F|          2|        1|         RJ|       405|            440|         SÃO GONÇALO|  7|739|
|      F|          3|        1|         PE|       435|            360|           ARCOVERDE|  9|667|
|      F|          1|        3|         SE|       457|            940|NOSSA SENHORA DA ...| 11|587|
|      F|          3|        1|         SP|       536|            640|       CARAGUATATUBA| 17|759|
|      F|          3|        1|         PE|       463|              0|             CARPINA| 20|680|


In [40]:
### Nova contagem do Dataframe
df.count()

685952

In [41]:
### Visualização dos tipos de dados
df.printSchema()

root
 |-- TP_SEXO: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- SG_UF_PROVA: string (nullable = true)
 |-- NU_NOTA_MT: string (nullable = true)
 |-- NU_NOTA_REDACAO: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- id: long (nullable = false)
 |-- IDH: string (nullable = true)



In [42]:
### Correção do tipo de dados para campos quantitativos
df = df.withColumn("NU_NOTA_MT",df.NU_NOTA_MT.cast('int'))
df = df.withColumn("NU_NOTA_REDACAO",df.NU_NOTA_REDACAO.cast('int'))
df = df.withColumn("IDH",df.IDH.cast('int'))

In [43]:
df.printSchema()

root
 |-- TP_SEXO: string (nullable = true)
 |-- TP_COR_RACA: string (nullable = true)
 |-- TP_ESCOLA: string (nullable = true)
 |-- SG_UF_PROVA: string (nullable = true)
 |-- NU_NOTA_MT: integer (nullable = true)
 |-- NU_NOTA_REDACAO: integer (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- id: long (nullable = false)
 |-- IDH: integer (nullable = true)



## **Análise de Dados**

In [46]:
### Visualizar a nota de Matemática e Redação por ID
df.select("id", "NU_NOTA_REDACAO", "NU_NOTA_MT").show()

+---+---------------+----------+
| id|NU_NOTA_REDACAO|NU_NOTA_MT|
+---+---------------+----------+
|  2|            760|       565|
|  3|            320|       416|
|  7|            440|       405|
|  9|            360|       435|
| 11|            940|       457|
| 17|            640|       536|
| 20|              0|       463|
| 22|            640|       343|
| 23|            760|       625|
| 25|            520|       383|
| 27|            640|       573|
| 29|            300|       615|
| 30|            640|       745|
| 31|            360|       352|
| 33|            760|       536|
| 36|            540|       446|
| 37|            360|       475|
| 38|            420|       409|
| 39|            460|       568|
| 40|            560|       430|
+---+---------------+----------+
only showing top 20 rows



In [50]:
 ### Verificar a média das notas de Redação por Estado
(
    df
    .groupby("SG_UF_PROVA")
    .agg({'NU_NOTA_REDACAO': 'avg'})
    .show()
)

+-----------+--------------------+
|SG_UF_PROVA|avg(NU_NOTA_REDACAO)|
+-----------+--------------------+
|         SC|    656.468496657659|
|         RO|   615.1662143826323|
|         PI|   625.3053159188395|
|         AM|   557.1723036725214|
|         RR|   589.0848026868179|
|         GO|   634.6132377769355|
|         TO|   620.7722734978339|
|         MT|    615.506726457399|
|         SP|   653.1452776141235|
|         ES|   658.1747186279764|
|         PB|   632.0064464141822|
|         RS|   649.2176329075322|
|         MS|   613.2953443258972|
|         AL|   620.4586894586895|
|         MG|   659.2294578929424|
|         PA|   604.5131512647384|
|         BA|   618.1288262325726|
|         SE|   641.7946222168147|
|         PE|   627.2233967582805|
|         CE|    600.173394579138|
+-----------+--------------------+
only showing top 20 rows



In [51]:
 ### Verificar a média das notas de Matemática por Estado
(
    df
    .groupby("SG_UF_PROVA")
    .agg({'NU_NOTA_MT': 'avg'})
    .show()
)

+-----------+-----------------+
|SG_UF_PROVA|  avg(NU_NOTA_MT)|
+-----------+-----------------+
|         SC| 562.675010667046|
|         RO|513.9820217096336|
|         PI|513.8934888428358|
|         AM|489.6543039628393|
|         RR| 510.639798488665|
|         GO|529.9991386142025|
|         TO|514.3433791674515|
|         MT|523.4001793721973|
|         SP|564.0960932897223|
|         ES|553.9241252584029|
|         PB|517.3159145850121|
|         RS|550.6815640565175|
|         MS| 524.107056256062|
|         AL|516.7460826210826|
|         MG| 558.399249914763|
|         PA|493.0159427592462|
|         BA|510.0565931926162|
|         SE|512.8871517216206|
|         PE|529.5147286821706|
|         CE|521.1019833906245|
+-----------+-----------------+
only showing top 20 rows



In [52]:
### Verificar as notas mais altas de Redação por Estado
(
    df
    .groupby("SG_UF_PROVA")
    .agg({'NU_NOTA_REDACAO': 'max'})
    .show()
)

+-----------+--------------------+
|SG_UF_PROVA|max(NU_NOTA_REDACAO)|
+-----------+--------------------+
|         SC|                 980|
|         RO|                 980|
|         PI|                 980|
|         AM|                 980|
|         RR|                 980|
|         GO|                 980|
|         TO|                 980|
|         MT|                 980|
|         SP|                 980|
|         ES|                 980|
|         PB|                1000|
|         RS|                 980|
|         MS|                 980|
|         AL|                 980|
|         MG|                 980|
|         PA|                 980|
|         BA|                 980|
|         SE|                 980|
|         PE|                 980|
|         CE|                 980|
+-----------+--------------------+
only showing top 20 rows



In [53]:
### Verificar a média das notas agrupadas por Estado e Gênero
(
    df
    .groupby("SG_UF_PROVA", "TP_SEXO")
    .agg({'NU_NOTA_REDACAO': 'avg'})
    .show()
)

+-----------+-------+--------------------+
|SG_UF_PROVA|TP_SEXO|avg(NU_NOTA_REDACAO)|
+-----------+-------+--------------------+
|         SC|      M|   635.2116203355873|
|         RS|      M|   630.8541100962725|
|         AC|      M|    598.601856908793|
|         SC|      F|   664.8982024034165|
|         RR|      F|   597.6498800959232|
|         SP|      F|   661.3769013088079|
|         MA|      F|   611.8474617439883|
|         SE|      M|   629.6975153042852|
|         MA|      M|   600.1637242331591|
|         RO|      F|             624.825|
|         RN|      F|          631.503125|
|         CE|      M|   571.2987672226251|
|         PE|      M|    612.258064516129|
|         MT|      F|   621.6622691292876|
|         AL|      F|   628.1102702702702|
|         AM|      M|   546.6987951807229|
|         RR|      M|   569.0756302521008|
|         PR|      F|   645.4708316798004|
|         PI|      F|   628.6266531027467|
|         SP|      M|   634.8924766324596|
+----------

In [54]:
### Verificar a média das notas de Redação por Gênero para o Estado de Mato Grosso
(
    df
    .where(df.SG_UF_PROVA == 'MT')
    .groupby("SG_UF_PROVA", "TP_SEXO")
    .agg({'NU_NOTA_REDACAO': 'avg'})
    .show()
)

+-----------+-------+--------------------+
|SG_UF_PROVA|TP_SEXO|avg(NU_NOTA_REDACAO)|
+-----------+-------+--------------------+
|         MT|      F|   621.6622691292876|
|         MT|      M|    602.436974789916|
+-----------+-------+--------------------+



In [354]:
### Estatística Geral
(
    df
    .describe()
    .show()
)

+-------+-------+------------------+------------------+-----------+-----------------+------------------+------------+--------------------+-----------------+
|summary|TP_SEXO|       TP_COR_RACA|         TP_ESCOLA|SG_UF_PROVA|       NU_NOTA_MT|   NU_NOTA_REDACAO|NO_MUNICIPIO|                  id|              IDH|
+-------+-------+------------------+------------------+-----------+-----------------+------------------+------------+--------------------+-----------------+
|  count| 685952|            685952|            685952|     685952|           685952|            685952|      685952|              685952|           685952|
|   mean|   NULL| 2.091609908565031|1.5135854986937862|       NULL|529.0973260519687|  627.949623297257|        NULL|3.8937656996813393E9|712.5227173329912|
| stddev|   NULL|1.0120840035409129|0.6030645950685177|       NULL|112.4939917892006|204.18875102309443|        NULL|4.2761420428695874E9|76.82448949871981|
|    min|      F|                 0|                 1|   

## **Montagem do Dataframe para o Modelo**

In [57]:
df.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id|IDH|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2|559|
|      M|          3|        1|         ES|       416|            320|           CARIACICA|  3|718|
|      F|          2|        1|         RJ|       405|            440|         SÃO GONÇALO|  7|739|
|      F|          3|        1|         PE|       435|            360|           ARCOVERDE|  9|667|
|      F|          1|        3|         SE|       457|            940|NOSSA SENHORA DA ...| 11|587|
|      F|          3|        1|         SP|       536|            640|       CARAGUATATUBA| 17|759|
|      F|          3|        1|         PE|       463|              0|             CARPINA| 20|680|


In [58]:
### Execução dos 'String Indexer'
tp_sexo_index = StringIndexer(inputCol="TP_SEXO", outputCol="INDEXED_TP_SEXO")
model_tp_sexo = tp_sexo_index.fit(df)
df = model_tp_sexo.transform(df)

tp_cor_raca_index = StringIndexer(inputCol="TP_COR_RACA", outputCol="INDEXED_TP_COR_RACA")
model_tp_cor_raca = tp_cor_raca_index.fit(df)
df = model_tp_cor_raca.transform(df)

tp_escola_index = StringIndexer(inputCol="TP_ESCOLA", outputCol="INDEXED_TP_ESCOLA")
model_tp_escola = tp_escola_index.fit(df)
df = model_tp_escola.transform(df)

sg_uf_index = StringIndexer(inputCol="SG_UF_PROVA", outputCol="INDEXED_SG_UF_PROVA")
model_sg_uf = sg_uf_index.fit(df)
df = model_sg_uf.transform(df)

no_municipio_index = StringIndexer(inputCol="NO_MUNICIPIO", outputCol="INDEXED_NO_MUNICIPIO")
model_no_municipio = no_municipio_index.fit(df)
df = model_no_municipio.transform(df)

In [59]:
### Aplicação do 'Hot Encoder'
encoders = OneHotEncoder(
    inputCols=["INDEXED_TP_SEXO", "INDEXED_TP_COR_RACA", "INDEXED_TP_ESCOLA", "INDEXED_SG_UF_PROVA", "INDEXED_NO_MUNICIPIO"],
    outputCols=["TP_SEXO_VEC", "TP_COR_RACA_VEC", "TP_ESCOLA_VEC", "SG_UF_PROVA_VEC", "NO_MUNICIPIO_VEC"],
    dropLast=False
    )

In [60]:
df = encoders.fit(df).transform(df)
df.show()

+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+---------------+-------------------+-----------------+-------------------+--------------------+-------------+---------------+-------------+---------------+-------------------+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|        NO_MUNICIPIO| id|IDH|INDEXED_TP_SEXO|INDEXED_TP_COR_RACA|INDEXED_TP_ESCOLA|INDEXED_SG_UF_PROVA|INDEXED_NO_MUNICIPIO|  TP_SEXO_VEC|TP_COR_RACA_VEC|TP_ESCOLA_VEC|SG_UF_PROVA_VEC|   NO_MUNICIPIO_VEC|
+-------+-----------+---------+-----------+----------+---------------+--------------------+---+---+---------------+-------------------+-----------------+-------------------+--------------------+-------------+---------------+-------------+---------------+-------------------+
|      F|          2|        1|         BA|       565|            760|PRESIDENTE TANCRE...|  2|559|            0.0|                2.0|              0.0|                2.0|  

## **Montando o vetor de features**

In [61]:
assembler = VectorAssembler(inputCols=["TP_SEXO_VEC", "TP_COR_RACA_VEC", "TP_ESCOLA_VEC", "SG_UF_PROVA_VEC", "NU_NOTA_MT", "NU_NOTA_REDACAO", "NO_MUNICIPIO_VEC", "IDH"], outputCol="features")
df_features = assembler.transform(df)

In [62]:
### Visualização do resultado das 'features'
df_features.select(df_features.features).show(20, False)

+-------------------------------------------------------------------------+
|features                                                                 |
+-------------------------------------------------------------------------+
|(1567,[0,4,8,13,38,39,960,1566],[1.0,1.0,1.0,1.0,565.0,760.0,1.0,559.0]) |
|(1567,[1,2,8,28,38,39,131,1566],[1.0,1.0,1.0,1.0,416.0,320.0,1.0,718.0]) |
|(1567,[0,4,8,14,38,39,66,1566],[1.0,1.0,1.0,1.0,405.0,440.0,1.0,739.0])  |
|(1567,[0,2,8,16,38,39,259,1566],[1.0,1.0,1.0,1.0,435.0,360.0,1.0,667.0]) |
|(1567,[0,3,10,24,38,39,280,1566],[1.0,1.0,1.0,1.0,457.0,940.0,1.0,587.0])|
|(1567,[0,2,8,11,38,39,599,1566],[1.0,1.0,1.0,1.0,536.0,640.0,1.0,759.0]) |
|(1567,[0,2,8,16,38,320,1566],[1.0,1.0,1.0,1.0,463.0,1.0,680.0])          |
|(1567,[0,3,8,14,38,39,379,1566],[1.0,1.0,1.0,1.0,343.0,640.0,1.0,715.0]) |
|(1567,[1,2,8,12,38,39,42,1566],[1.0,1.0,1.0,1.0,625.0,760.0,1.0,754.0])  |
|(1567,[1,2,8,13,38,39,1448,1566],[1.0,1.0,1.0,1.0,383.0,520.0,1.0,557.0])|
|(1567,[1,4,

## **Executar o Modelo Não Supervisionado (Cluster) K-Means**

In [65]:
kmeans = KMeans().setK(3)
modeloK = kmeans.fit(df_features)
df_agrupado = modeloK.transform(df_features)
df_agrupado.show(truncate=False)

+-------+-----------+---------+-----------+----------+---------------+-------------------------+---+---+---------------+-------------------+-----------------+-------------------+--------------------+-------------+---------------+-------------+---------------+-------------------+-------------------------------------------------------------------------+----------+
|TP_SEXO|TP_COR_RACA|TP_ESCOLA|SG_UF_PROVA|NU_NOTA_MT|NU_NOTA_REDACAO|NO_MUNICIPIO             |id |IDH|INDEXED_TP_SEXO|INDEXED_TP_COR_RACA|INDEXED_TP_ESCOLA|INDEXED_SG_UF_PROVA|INDEXED_NO_MUNICIPIO|TP_SEXO_VEC  |TP_COR_RACA_VEC|TP_ESCOLA_VEC|SG_UF_PROVA_VEC|NO_MUNICIPIO_VEC   |features                                                                 |prediction|
+-------+-----------+---------+-----------+----------+---------------+-------------------------+---+---+---------------+-------------------+-----------------+-------------------+--------------------+-------------+---------------+-------------+---------------+-----------